In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 75kB/s 
     |████████████████████████████████| 204kB 21.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=faf90ec7117657f247edf466e3cb5a4b48cd3836382aa0ac11b0a9f5995a2608
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [4]:
spark = SparkSession.builder.appName('sparkify') \
    .config('spark.driver.maxResultSize', '3g') \
    .getOrCreate()

In [5]:
df = spark.read.parquet("/content/drive/MyDrive/datasets/dsnd-sparkify/event_labeled.parquet")
df = df.drop('userIdTemp')
df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- up_ts: timestamp (nullable = true)
 |-- down_ts: string (nullable = true)
 |-- isChurn: boolean (nullable = true)
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [85]:
def change_colname_join_df(join_df, suffix='_temp'):
    '''
    INPUT:
    join_df - dataframe on the right side of join
    suffix - added string on each column name

    OUTPUT:
    res_df - dataframe with renamed column
    '''
    res_df = join_df
    for col_name in join_df.columns:
        res_df = res_df.withColumnRenamed(col_name, col_name + suffix)
    
    return res_df

def chain_and(df1, df2, cols, suffix="_temp"):
    for i, col in enumerate(cols):
        if i == 0:
            res = df1[col] == df2[col + suffix]
        else:
            res = res & (df1[col] == df2[col + suffix])
    
    return res

# def quick_left_join(df1, dfs, key_cols):
#     res_df = df1

#     for df_temp in dfs:
#         res_df.join

In [87]:
jdf = change_colname_join_df(n_songs_play_df)
df.limit(50).join(jdf, chain_and(df, jdf, ["userId", "up_ts"])).show()

+-------+-------------------+-------------------+-------+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+--------------------+-------------------+-----------+-------------------+------------+
| userId|              up_ts|            down_ts|isChurn|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           userAgent|                 ts|userId_temp|         up_ts_temp|n_songs_temp|
+-------+-------------------+-------------------+-------+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+--------------------+-------------------+-----------+-------------------+------------+
|1006381|2018-10-30

In [77]:
for i, col in enumerate(["userId", "ts"]):
    if i == 0:
        r = df[col] == df[col]
    else:
        r = r & (df["ts"] == df["ts"])

r 

Column<'((userId = userId) AND (ts = ts))'>

In [65]:
# number of song heard in one subscription
n_songs_play_df = df.filter(df.page == "NextSong").groupBy(["userId", "up_ts"]).count().withColumnRenamed("count", "n_songs")
n_songs_play_df.show(5)

+-------+-------------------+-------+
| userId|              up_ts|n_songs|
+-------+-------------------+-------+
|1367536|2018-10-08 14:58:27|   1806|
|1322258|2018-10-08 23:28:14|    997|
|1554972|2018-11-19 21:16:01|    472|
|1655008|2018-10-25 06:50:45|    234|
|1968237|2018-10-27 11:19:26|    700|
+-------+-------------------+-------+
only showing top 5 rows



In [16]:
# number of day in subscription
maxdate_df = df.select("ts").agg(F.max(df.ts))

datediff_df = df.select(["userId", "up_ts", "down_ts", "isChurn"]).dropDuplicates() \
    .join(maxdate_df, ~df.userId.isNull(), how='left') \
    .withColumn("datediff", 
        F.datediff(F.when(F.col("isChurn"), F.col("down_ts")).otherwise(F.col("max(ts)").cast(T.TimestampType())), df.up_ts)) \
    .drop("max(ts)")

datediff_df.show(5)

+-------+-------------------+-------------------+-------+--------+
| userId|              up_ts|            down_ts|isChurn|datediff|
+-------+-------------------+-------------------+-------+--------+
|1111091|2018-11-27 17:43:31|2099-12-31 00:00:00|  false|       3|
|1161080|2018-10-26 19:14:14|2018-11-23 21:55:06|   true|      28|
|1291366|2018-10-01 04:22:41|2099-12-31 00:00:00|  false|      60|
|1335330|2018-11-26 15:00:33|2099-12-31 00:00:00|  false|       4|
|1721316|2018-10-11 17:19:44|2099-12-31 00:00:00|  false|      50|
+-------+-------------------+-------------------+-------+--------+
only showing top 5 rows



In [13]:
# number of song played per day
jdf = change_colname_join_df(datediff_df)
song_rate = df.filter(df.page == 'NextSong').groupBy(["userId", "up_ts"]).count() \
    .withColumnRenamed("count", "song_count") \
    .join(jdf, (df.up_ts == jdf.up_ts_temp) & (df.userId == jdf.userId_temp)) \
    .drop("userId_temp", "up_ts_temp", "down_ts_temp", "isChurn_temp") \
    .withColumn("song_rate", F.col("song_count") / F.when(F.col("datediff_temp") == 0, 1).otherwise(F.col("datediff_temp")))

song_rate.show(5)

+-------+-------------------+----------+-------------+------------------+
| userId|              up_ts|song_count|datediff_temp|         song_rate|
+-------+-------------------+----------+-------------+------------------+
|1071843|2018-11-08 13:16:59|      1190|           22| 54.09090909090909|
|1120784|2018-10-11 14:16:28|      1502|           50|             30.04|
|1128522|2018-11-13 17:46:22|       203|            2|             101.5|
|1130061|2018-10-11 20:04:50|        96|           50|              1.92|
|1135039|2018-10-04 11:31:06|       643|           57|11.280701754385966|
+-------+-------------------+----------+-------------+------------------+
only showing top 5 rows



In [27]:
# number of songs added to playlist
n_songs_df = df.select(["userId", "up_ts", "page"]).filter(df.page =="Add to Playlist") \
    .groupBy(["userId", "up_ts"]) \
    .agg(F.count(F.col("page")).alias("n_song"))

n_songs_df.show(5)

+-------+-------------------+------+
| userId|              up_ts|n_song|
+-------+-------------------+------+
|1367536|2018-10-08 14:58:27|    39|
|1322258|2018-10-08 23:28:14|    30|
|1554972|2018-11-19 21:16:01|    21|
|1655008|2018-10-25 06:50:45|    11|
|1968237|2018-10-27 11:19:26|    15|
+-------+-------------------+------+
only showing top 5 rows



In [50]:
#number of thumbs up and down
tup_tdown_df = df.select(["userId", "up_ts", "down_ts", "isChurn", "page"]).filter(df.page.isin(["Thumbs Up", "Thumbs Down"])) \
    .groupby(["userId", "up_ts", "page"]) \
    .agg(F.count(F.col("page"))) \
    .groupby(["userId", "up_ts"]) \
    .pivot("page") \
    .agg(F.first("count(page)"))

tup_tdown_df.show(5)


+-------+-------------------+-----------+---------+
| userId|              up_ts|Thumbs Down|Thumbs Up|
+-------+-------------------+-----------+---------+
|1427546|2018-10-17 22:52:00|          4|       25|
|1712775|2018-11-23 17:56:01|          6|       35|
|1358238|2018-10-02 14:25:59|         23|       64|
|1071843|2018-11-08 13:16:59|          6|       51|
|1554972|2018-11-19 21:16:01|          5|       35|
+-------+-------------------+-----------+---------+
only showing top 5 rows



In [18]:
# average session length and number of session
session_df = df.groupBy(["userId", "up_ts", "sessionId"]) \
    .agg(
        F.min(df.ts).cast(T.LongType()).alias("min"),
        F.max(df.ts).cast(T.LongType()).alias("max")
    ) \
    .withColumn("diff", (F.col("max") - F.col("min"))) \
    .groupBy(["userId", "up_ts"]) \
    .agg(F.avg(F.col("diff")).alias("avg_sess_len"), F.count(F.col("sessionId")).alias("sess_count"))

session_df.show(5)

+-------+-------------------+------------------+----------+
| userId|              up_ts|      avg_sess_len|sess_count|
+-------+-------------------+------------------+----------+
|1914133|2018-10-09 11:11:55|           22340.0|        12|
|1071843|2018-11-08 13:16:59|27020.272727272728|        11|
|1582360|2018-10-04 15:48:38|19012.117647058825|        34|
|1576394|2018-10-03 07:51:46| 19205.60606060606|        33|
|1957517|2018-10-11 04:46:39|          28964.75|        12|
+-------+-------------------+------------------+----------+
only showing top 5 rows



In [58]:
# platform for each user
device_df = df.select(["userId", "up_ts", "userAgent"]).withColumn("platform",
    F.when(df.userAgent.contains("Macintosh"), "macos") \
    .when(df.userAgent.contains("Windows"), "windows") \
    .when(df.userAgent.contains("iPad"), "ipad") \
    .when(df.userAgent.contains("iPhone"), "iphone") \
    .when(df.userAgent.contains("Linux"), "linux")) \
    .groupBy(["userId", "up_ts", "platform"]) \
    .agg(F.count("platform")) \
    .withColumn("isplatform", ~F.isnull(F.col("count(platform)"))) \
    .groupby(["userId", "up_ts"]) \
    .pivot("platform") \
    .agg(F.max("isplatform")) \
    .fillna(False)

device_df.show(5)

+-------+-------------------+-----+------+-----+-----+-------+
| userId|              up_ts| ipad|iphone|linux|macos|windows|
+-------+-------------------+-----+------+-----+-----+-------+
|1130061|2018-10-11 20:04:50|false| false|false| true|  false|
|1914133|2018-10-09 11:11:55|false| false|false| true|  false|
|1128522|2018-11-13 17:46:22|false| false|false| true|  false|
|1712775|2018-11-23 17:56:01|false| false|false|false|   true|
|1568402|2018-10-18 14:26:47|false|  true|false|false|  false|
+-------+-------------------+-----+------+-----+-----+-------+
only showing top 5 rows



In [64]:
#state
state_df = df.select(["userId","up_ts","location"]) \
    .withColumn("state", F.split(df.location, ', ')[1]) \
    .groupBy(["userId", "up_ts", "state"]) \
    .agg(~F.isnull(F.count(F.col("state")))) \
    .withColumnRenamed("(NOT (count(state) IS NULL))", "isstate") \
    .groupBy(["userId", "up_ts"]) \
    .pivot("state") \
    .agg(F.max("isstate")) \
    .fillna(False)

state_df.show(5)

+-------+-------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+-----+-----+-----+-----+-----+-----+-----+-----+--------+--------+-----+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----------+-----+-----+-----+-----+-----+-----+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+-----+-----+-----+
| userId|              up_ts|   AK|   AL|   AR|AR-MO|AR-OK|   AZ|   CA|   CO|   CT|DC-VA-MD-WV|   DE|   FL|   GA|GA-AL|GA-SC|   HI|   IA|IA-IL|IA-IL-MO|IA-NE-SD|   ID|   IL|IL-IN-WI|IL-MO|   IN|IN-KY|IN-MI|   KS|   KY|KY-IL|KY-IN|   LA|   MA|MA-CT|MA-NH|   MD|MD-DE|MD-WV|   ME|   MI|MI-WI|   MN|MN-WI|   MO|MO-IL|MO-KS|   MS|MS-LA|   MT|   NC